In [1]:
import math as mth
import json
import os
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
import glob
import sys
import json
import time
import copy
import torch
import click
import pickle
import numpy as np
import logging as log
import cv2
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import albumentations as A
import torch, torch.nn as nn
import segmentation_models_pytorch as sm

from tqdm import tqdm
from pathlib import Path
from skimage.io import imread
from torch.autograd import Variable
from losses_pytorch.segmentation.twodim.entropy_based_losses import cross_entropy_with_logits_loss

from precode.models import UnetSm
from precode.scoring import jaccard_score
from precode.labels.wheat_segmentation import WHEAT_LABELS
from precode.net_utils import init_determenistic, batch_ids_generator

In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mpl

In [21]:
file_path = '/home/jupyter-fly_cv/Igor/Plants/'
file_path_2 = '/home/jupyter-fly_cv/Igor/Plants_2/Segment/val/for_val_Pl_2'
file_path_3 = '/home/jupyter-fly_cv/Igor/Plants/New_json/train'
file_path_4 = '/home/jupyter-fly_cv/Igor/Plants/New_json/test'
model_checker_path = '/home/jupyter-fly_cv/Igor/Plants/model_segmentation_checker.bin'

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

infer_aug = A.Compose([ 
                        A.Resize ( height=512,
                                   width=512,
                                   interpolation=1,
                                   always_apply=False,
                                   p=1. ),
                        A.Normalize ( mean=(0.485, 0.456, 0.406),
                                      std=(0.229, 0.224, 0.225),
                                      max_pixel_value=255.0,
                                      always_apply=False,
                                      p=1.0 )
                      ])


class UnetSm(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, **kwargs):
        super().__init__()

        self.unet = sm.Unet(in_channels=in_channels, classes=out_channels, **kwargs)

    def forward(self, inputs):
        return self.unet(inputs)

def inverse_infer_aug(height, width):
    return A.Compose([
                       A.Resize ( height=height,
                                  width=width,
                                  interpolation=cv2.INTER_NEAREST,
                                  always_apply=False,
                                  p=1. ) 
                    ])

    
def create_model(model_file):
    model = UnetSm( out_channels=2,
                    encoder_name='efficientnet-b0' )
 
    with open(model_file, 'rb') as f:
        state = pickle.load(f)

    model.load_state_dict(state)
    model.cuda()

    return model


def torch_float(data, device):
    return Variable(torch.FloatTensor(data)).to(device)

def augmented_load(img, aug):
    auged = aug(image=img)
    aug_img = auged['image']

    aug_img = aug_img.transpose(2, 0, 1)

    return np.array([aug_img])

def infer(model, img):
    
    imgs_batch_ = augmented_load(img, infer_aug)
    imgs_batch = torch_float(imgs_batch_, torch.device('cuda'))

    logits_batch = model(imgs_batch)

    pred_masks_batch = logits_batch.argmax(dim=1)
    pred_mask = pred_masks_batch.cpu().data.numpy()#[0]
      #  print(pred_masks_batch.cpu().data.numpy().shape)

    original_size = img.shape[:2]

    original_pred_mask = inverse_infer_aug(*original_size)(image=pred_mask)['image']
    original_pred_mask = original_pred_mask.astype('uint8') * 255 

    return original_pred_mask

In [4]:
def read(name):
    file = open(name,'r')
    str_file = file.read()
    j = json.loads(str_file)
    file.close()
    return j

def square(poligon):
    sq = 0
    n = len(poligon)
    for i in range(0,n-1):
        sq += poligon[i][0]*poligon[i+1][1]
    sq +=poligon[n-1][0]*poligon[0][1]
    for i in range(0,n-1):
        sq -= poligon[i+1][0]*poligon[i][1]
    sq -=poligon[0][0]*poligon[n-1][1]
    return mth.fabs(sq)/2

def all_json_img(imgs_path):
    img_names = []

    for dirname, _, filenames in os.walk(imgs_path):
        for filename in filenames: 
            name = os.path.join(filename)
            if name.split('.')[-1] != 'json':
                continue
            img_names.append(dirname + '/' + name)
            
    return img_names

In [17]:
def all_jpg_img(imgs_path):
    img_names = []

    for dirname, _, filenames in os.walk(imgs_path):
        for filename in filenames: 
            name = os.path.join(filename)
            if name.split('.')[-1] != 'jpg':
                continue
            img_names.append(dirname + '/' + name)
            
    return img_names

In [5]:
def poligons_square(poligons):
    sq = 0
    for poligon in poligons:
        sq += square(poligon['points'])
    return sq

In [6]:
def color_checker(img):
    img_checker = infer(create_model(model_checker_path),img)
    img_checker = cv2.GaussianBlur(img_checker, (21, 21), 0)
    img_checker = 1 - np.round(img_checker/255)
    return img_checker

In [7]:
def checker_mass():
    checker = infer(create_model(model_checker_path),img)
    checker_count = 0
    for i in range(checker.shape[0]):
        for j in range(checker.shape[1]):
            if checker[i][j] == 0:
                checker_count += 1
    return checker_count

In [8]:
def create_table_with_B_mass(b_mass):
    df = pd.DataFrame(b_mass,columns = ['Name', 'Bio_Mass'])
    df.to_csv('bio_mass_json.csv', index = False)

In [9]:
def create_mask(poligons, height, width):
    mask = np.zeros((height,width),dtype=np.uint8)
    for poligon in poligons:
        cv2.fillPoly(mask,np.array([poligon['points']],dtype = np.int32), 255)
    return mask    

In [10]:
def filename_suff_png(name, suff):
    names = name.split('/')
    png_names = names[-1].split('.')
    names[-1] = png_names[0] + suff +'.' + 'png'
    new_name = ''
    for n in names:
        if n != names[-1]:
            new_name =new_name + n + '/'
        else:
            new_name =new_name + n
    return new_name    

In [15]:
def rename_JPG(name):
    names = name.split('.')
    return names[0] + '.JPG'

In [16]:
 rename_JPG('/home/jupyter-fly_cv/Igor/Plants_2/Segment/train/101MEDIA_DJI_0655.jpg')

'/home/jupyter-fly_cv/Igor/Plants_2/Segment/train/101MEDIA_DJI_0655.JPG'

In [22]:
file_names = all_json_img(file_path_2)
#b_mass = []
for name in file_names:
    #name = '/home/jupyter-fly_cv/Igor/Plants_2/Segment/train/101MEDIA_DJI_0655.jpg'
    #plt.imshow(img)
    poligons = read(name) 
    #b_m = poligons_square(poligons['shapes'])/(poligons['imageHeight']*poligons['imageWidth'])
    #b_m = b_m/checker_mass()
    #b_mass.append([name.split('/')[-1].split('.')[0],b_m])
    mask = create_mask(poligons['shapes'],poligons['imageHeight'],poligons['imageWidth'])
    mask_name = filename_suff_png(name, '')
    cv2.imwrite(mask_name,mask)
    
#create_table_with_B_mass(b_mass)

In [23]:
file_names = all_jpg_img(file_path_2)
#b_mass = []
for name in file_names:
    os.rename(name, rename_JPG(name))

In [12]:
!nvidia-smi

Mon Apr 11 11:54:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:09:00.0 Off |                  N/A |
| 27%   40C    P8    25W / 250W |   9391MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:0A:00.0 Off |                  N/A |
| 27%   